In [2]:
import h5py
# 

Convert D-FAUST dataset into .ply file

In [15]:
file_path = "/usr/stud/srinivaa/storage/slurm/cadex/dfaust_resgistered/Humans/50004_hips/mesh_seq_registered/00000000.npz"
data = np.load(file_path)
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(data['points'])
# o3d.io.write_point_cloud("registered_dfaust.ply", pcd)

o3d_mesh = o3d.geometry.TriangleMesh()
o3d_mesh.vertices = o3d.utility.Vector3dVector(data["points"]) # verify what is the name of the attribute for vertices in data dictionary. It should be vertices as given in load method in MeshField class.
o3d_mesh.triangles = o3d.utility.Vector3iVector(data["triangles"])
o3d.io.write_triangle_mesh("registered_dfaust.ply",o3d_mesh)

True

### Extract data from registred DFAUST scans

In [5]:
filename = "/usr/stud/srinivaa/storage/slurm/cadex/dfaust_resgistered/registrations_f.hdf5"
f =  h5py.File(filename, "r")
a_dataset_keys = list(f.keys()) #extract the different groups of meshes    


In [6]:
dataset_list = [a_dataset_keys[0],a_dataset_keys[-1]]

In [7]:

for data in dataset_list:
    data_face = f[data]
data_face_arr = np.asarray(data_face)

for data in a_dataset_keys[:1]:
    data_i = f[data]
data_i_arr = np.asarray(data_i)

In [12]:
file_index = "%08d"%9
file_index

'00000009'

In [65]:
o3d_mesh = o3d.geometry.TriangleMesh()
o3d_mesh.vertices = o3d.utility.Vector3dVector(data_i_arr[:,:,0]) # verify what is the name of the attribute for vertices in data dictionary. It should be vertices as given in load method in MeshField class.
o3d_mesh.triangles = o3d.utility.Vector3iVector(data_face_arr)
o3d.io.write_triangle_mesh("registered_dfaust.ply",o3d_mesh)

True

## Custom dataset of CaDeX

In [2]:
import numpy as np
import os
import dataset.oflow_dataset as oflow_dataset
from torch.utils import data
from torch.utils.data import DataLoader

Write a class for custom dataset 

In [3]:
class HumansDataset(data.Dataset):
    def __init__(
        self,
        dataset_folder,
        fields,
        split=None,
        categories=None,
        no_except=True,
        transform=None,
        length_sequence=17,
        n_files_per_sequence=-1,
        offset_sequence=0,
        ex_folder_name="pcl_seq",
        **kwargs
    ):
        # Attributes
        self.dataset_folder = dataset_folder
        self.fields = fields
        self.no_except = no_except
        self.transform = transform
        self.length_sequence = length_sequence
        self.n_files_per_sequence = n_files_per_sequence
        self.offset_sequence = offset_sequence
        self.ex_folder_name = ex_folder_name
       
        # Read metadata file
        metadata_file = os.path.join(dataset_folder, "metadata.yaml")

        
        self.metadata = {c: {"id": c, "name": "n/a"} for c in categories}

        # Set index
        for c_idx, c in enumerate(categories):
            self.metadata[c]["idx"] = c_idx #only one category: D-FAUST. contains single ID only

        # Get all models
        self.models = []
        for c_idx, c in enumerate(categories):
            subpath = os.path.join(dataset_folder, c) #subpath: /usr/stud/srinivaa/code/new_CaDeX/CaDeX/resource/data/Humans/D-FAUST
           
            if split is not None and os.path.exists(os.path.join(subpath, split + ".lst")):
                split_file = os.path.join(subpath, split + ".lst") # for train mode: /usr/stud/srinivaa/code/new_CaDeX/CaDeX/resource/data/Humans/D-FAUST/train.lst
                with open(split_file, "r") as f:
                    models_c = f.read().split("\n") # All files in train.lst for training mode
           
            models_c = list(filter(lambda x: len(x) > 0, models_c))
            models_len = self.get_models_seq_len(subpath, models_c) # gives the total number .npz files in each model
            models_c, start_idx = self.subdivide_into_sequences(models_c, models_len)
            self.models += [
                {"category": c, "model": m, "start_idx": start_idx[i]}
                for i, m in enumerate(models_c)
            ]
        
       

    def __len__(self):
       return len(self.models)
    
    
    def __getitem__(self,idx):
        category = self.models[idx]["category"]
        model = self.models[idx]["model"]
        start_idx = self.models[idx]["start_idx"]
        c_idx = self.metadata[category]["idx"]

        model_path = os.path.join(self.dataset_folder, category, model)

        data = {}

        
        for field_name, field in self.fields.items():
            field_data = field.load(model_path, idx, c_idx, start_idx)
 
            if isinstance(field_data, dict):
                for k, v in field_data.items():
                    if k is None:
                        data[field_name] = v
                    else:
                        data["%s.%s" % (field_name, k)] = v
            else:
                data[field_name] = field_data
           

        if self.transform is not None:
            data = self.transform(data)

        
        
        return data

    def get_models_seq_len(self, subpath, models):
        """Returns the sequence length of a specific model.

        This is a little "hacky" as we assume the existence of the folder
        self.ex_folder_name. However, in our case this is always given.

        Args:
            subpath (str): subpath of model category
            models (list): list of model names
        """
        ex_folder_name = self.ex_folder_name
        models_seq_len = []
        for m in models:
            _sublist = [
                f for f in os.listdir(os.path.join(subpath, m, ex_folder_name)) if "_" not in f
            ]
            models_seq_len.append(len(_sublist))
        # models_seq_len = [len(os.listdir(os.path.join(subpath, m, ex_folder_name))) for m in models]
        return models_seq_len

    def subdivide_into_sequences(self, models, models_len):
        """Subdivides model sequence into smaller sequences.

        Args:
            models (list): list of model names
            models_len (list): list of lengths of model sequences
        """
        length_sequence = self.length_sequence
        n_files_per_sequence = self.n_files_per_sequence
        offset_sequence = self.offset_sequence

        # Remove files before offset
        models_len = [l - offset_sequence for l in models_len]

        # Reduce to maximum number of files that should be considered
        if n_files_per_sequence > 0:
            models_len = [min(n_files_per_sequence, l) for l in models_len]

        models_out = []
        start_idx = []
        for idx, model in enumerate(models):
            for n in range(0, models_len[idx] - length_sequence + 1):
                models_out.append(model)
                start_idx.append(n + offset_sequence)

        return models_out, start_idx   
    

In [4]:
def get_transforms():
    """Returns transform objects.

    Args:
        cfg (yaml config): yaml config object
    """
    n_pcl = 100
    n_pt = 512
    n_pt_eval = 10000

    transf_pt = oflow_dataset.SubsamplePoints(n_pt)
    transf_pt_val = oflow_dataset.SubsamplePointsSeq(n_pt_eval, random=False)
    transf_pcl_val = oflow_dataset.SubsamplePointcloudSeq(n_pt_eval, random=False)
    transf_pcl = oflow_dataset.SubsamplePointcloudSeq(n_pcl, connected_samples=True)

    return transf_pt, transf_pt_val, transf_pcl, transf_pcl_val

In [5]:
def get_data_fields(mode):
    """Returns data fields.

    Args:
        mode (str): mode (train|val|test)
        cfg (yaml config): yaml config object
    """
    fields = {}
    seq_len_train = 17
   
    seq_len_val = seq_len_train
    p_folder = "points_seq" # points_seq: contains information regarding the points and their corresponding occupancy values
    pcl_folder = "pcl_seq" #pcl_seq : contains information regarding the points, scale and loc
    mesh_folder = "mesh_registred" #mesh_seq: non-existent. Utilize this to have a file containing points and faces for each model.
    generate_interpolate = False #False
    unpackbits = True # True
    
    training_all = False
    
    n_training_frames = 8

    # Transformation
    transf_pt, transf_pt_val, transf_pcl, transf_pcl_val = get_transforms()

    # Fields
    pts_iou_field = oflow_dataset.PointsSubseqField
    pts_corr_field = oflow_dataset.PointCloudSubseqField

    # MeshSubseqField can be used to load mesh fields

  
    not_choose_last = False
    training_multi_files = False
    
    loss_recon = "true"
    loss_corr = "true"

    if mode == "train":
        if loss_recon:
            if training_all:
                fields["points"] = pts_iou_field(
                    p_folder,
                    transform=transf_pt,
                    all_steps=True,
                    seq_len=seq_len_train,
                    unpackbits=unpackbits,
                    use_multi_files=training_multi_files,
                )
            else:
                fields["points"] = pts_iou_field(
                    p_folder,
                    sample_nframes=n_training_frames,
                    transform=transf_pt,
                    seq_len=seq_len_train,
                    fixed_time_step=0,
                    unpackbits=unpackbits,
                    use_multi_files=training_multi_files,
                )
            # fields["points_t"] = pts_iou_field(
            #     p_folder,
            #     transform=transf_pt,
            #     seq_len=seq_len_train,
            #     unpackbits=unpackbits,
            #     not_choose_last=not_choose_last,
            #     use_multi_files=training_multi_files,
            # )


            # fields["mesh"] = oflow_dataset.MeshField(
            # mesh_folder, seq_len=seq_len_val)

    # only training can be boost by multi-files
    # modify here, if not train, val should also load the same as the test
    else:
        fields["points"] = pts_iou_field(
            p_folder,
            transform=transf_pt_val,
            all_steps=True,
            seq_len=seq_len_val,
            unpackbits=unpackbits,
        )
        fields[
            "points_mesh"
        ] = pts_corr_field(  # ? this if for correspondence? Checked, this is for chamfer distance, make sure that because here we use tranforms, teh pts in config file must be 100000
            pcl_folder, transform=transf_pcl_val, seq_len=seq_len_val
        )
    # Connectivity Loss:
    if loss_corr:
        # fields["pointcloud"] = pts_corr_field(
        #     pcl_folder,
        #     transform=transf_pcl,
        #     seq_len=seq_len_train,
        #     use_multi_files=training_multi_files,
        # )
        fields["pointcloud"] = oflow_dataset.PointCloudField(
            mesh_folder, seq_len=seq_len_val)
    if mode == "test" and generate_interpolate:
        fields["mesh"] = oflow_dataset.MeshSubseqField(
            mesh_folder, seq_len=seq_len_val, only_end_points=True
        )
    fields["oflow_idx"] = oflow_dataset.IndexField()
    return fields

In [6]:
def get_inputs_field(mode):
    
    input_type = "mesh_seq"
    seq_len_train = 17
    seq_len_val = seq_len_train
    
    seq_len = seq_len_train
    

    if input_type is None:
        inputs_field = None
    elif input_type == "img_seq":
        if mode == "train" and cfg["dataset"]["oflow_config"]["img_augment"]:
            resize_op = transforms.RandomResizedCrop(
                cfg["dataset"]["oflow_config"]["img_size"], (0.75, 1.0), (1.0, 1.0)
            )
        else:
            resize_op = transforms.Resize((cfg["dataset"]["oflow_config"]["img_size"]))

        transform = transforms.Compose(
            [
                resize_op,
                transforms.ToTensor(),
            ]
        )

        if mode == "train":
            random_view = True
        else:
            random_view = False

        inputs_field = oflow_dataset.ImageSubseqField(
            cfg["dataset"]["oflow_config"]["img_seq_folder"], transform, random_view=random_view
        )
    elif input_type == "pcl_seq":
        connected_samples = cfg["dataset"]["oflow_config"]["input_pointcloud_corresponding"]
        transform = transforms.Compose(
            [
                oflow_dataset.SubsamplePointcloudSeq(
                    cfg["dataset"]["oflow_config"]["input_pointcloud_n"],
                    connected_samples=connected_samples,
                ),
                oflow_dataset.PointcloudNoise(
                    cfg["dataset"]["oflow_config"]["input_pointcloud_noise"]
                ),
            ]
        )
        training_multi_files = False
        if "training_multi_files" in cfg["dataset"]["oflow_config"]:
            if cfg["dataset"]["oflow_config"]["training_multi_files"] and mode == "train":
                training_multi_files = True
                logging.info(
                    "Oflow D-FAUST PCL Field use multi files to speed up disk performation"
                )

        inputs_field = oflow_dataset.PointCloudSubseqField(
            cfg["dataset"]["oflow_config"]["pointcloud_seq_folder"],
            transform,
            seq_len=seq_len,
            use_multi_files=training_multi_files,
        )
    #TODO : get inputs fields for mesh sequence
    elif input_type == "mesh_seq":
        
        # transform = transforms.Compose(
        #     [
        #         #oflow_dataset.MeshNoise(),
        #         #oflow_dataset.DownSampleMesh(N = 512)
        #     ]
        # )

        inputs_field = oflow_dataset.MeshField(
            "mesh_registred"
        )
    elif input_type == "end_pointclouds":
        transform = oflow_dataset.SubsamplePointcloudSeq(
            cfg["dataset"]["oflow_config"]["input_pointcloud_n"],
            connected_samples=cfg["dataset"]["oflow_config"]["input_pointcloud_corresponding"],
        )

        inputs_field = oflow_dataset.PointCloudSubseqField(
            cfg["dataset"]["oflow_config"]["pointcloud_seq_folder"],
            only_end_points=True,
            seq_len=seq_len,
            transform=transform,
        )
    elif input_type == "idx":
        inputs_field = oflow_dataset.IndexField()
    else:
        raise ValueError("Invalid input type (%s)" % input_type)
    return inputs_field

In [7]:
fields = get_data_fields("train")
inputs_field = get_inputs_field("train")

if inputs_field is not None:
    fields["inputs"] = inputs_field

In [8]:
dataset_folder = "/usr/data/cvpr_shared/marvin/Data/CaDeX/data/Humans"
categories = ["D-FAUST"]

Instantiate a custom dataset object

In [9]:
dataset = HumansDataset(
        dataset_folder,
        fields,
        split="train",
        categories=categories,
        length_sequence=17,
        n_files_per_sequence=-1,
        offset_sequence=15,
        ex_folder_name="mesh_registred",
    )

In [12]:
dataset[1]['inputs.vertices'].shape

(17, 6890, 3)

Test the dataloader

### Study shape class

In [12]:
from core.models.utils_arap.shape_utils import Shape
import torch

In [13]:
shape_1 = Shape(vert=torch.from_numpy(dataset[1]['mesh.vertices']),triv=torch.from_numpy(dataset[1]['mesh.triangles']))

### Convert .npz file into .ply file

In [5]:
# activate cadex_exp venv
import open3d as o3d
import numpy as np

In [23]:
vertices_path = "//usr/stud/srinivaa/code/new_CaDeX/CaDeX/cdc_1.npz"
data_vertices = np.load(file_path)
#data_faces = np.load("/usr/data/cvpr_shared/marvin/Data/CaDeX/data/Humans/D-FAUST/50002_chicken_wings/mesh_registred/00000000.npz")["triangles"]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.vstack(data_vertices['points']))
o3d.io.write_point_cloud("cdc_1.ply", pcd)

# o3d_mesh = o3d.geometry.TriangleMesh()
# o3d_mesh.vertices = o3d.utility.Vector3dVector(np.vstack(data_vertices["points"])) # verify what is the name of the attribute for vertices in data dictionary. It should be vertices as given in load method in MeshField class.
# o3d_mesh.triangles = o3d.utility.Vector3iVector(data_faces)
# o3d.io.write_triangle_mesh("cdc_0_mesh.ply",o3d_mesh)

True

### Study ARAP loss

In [33]:
from core.models.utils_arap.arap_interpolation import ArapInterpolationEnergy
import torch
import numpy as np
from core.models.utils_arap.shape_utils import Shape


In [143]:
def arap_exact(vert_diff_t, vert_diff_0, neigh, n_vert):

    #EXPLORE : Get the shapes of vert_diff_t and vert_diff_0. Check if they are tnesors and batch matrix multiplication is possible
    print("Shape of vert_diff_t:", vert_diff_t.shape)
    print("Shape of vert_diff_0:", vert_diff_0.shape)


    S_neigh = torch.bmm(vert_diff_t.unsqueeze(2), vert_diff_0.unsqueeze(1))

    S = my_zeros([n_vert, 3, 3])

    S = torch.index_add(S, 0, neigh[:, 0], S_neigh)
    S = torch.index_add(S, 0, neigh[:, 1], S_neigh)

    U, _, V = torch.svd(S.cpu(), compute_uv=True)

    U = U.to(device)
    V = V.to(device)

    R = torch.bmm(U, V.transpose(1, 2))

    Sigma = my_ones((R.shape[0], 1, 3))
    Sigma[:, :, 2] = torch.det(R).unsqueeze(1)

    R = torch.bmm(U * Sigma, V.transpose(1, 2))

    return R


def arap_energy_exact(vert_t, vert_0, neigh, lambda_reg_len=1e-6):
    n_vert = vert_t.shape[0]
    print("Neighbours:",neigh[:,0])
    vert_diff_t = vert_t[neigh[:, 0], :] - vert_t[neigh[:, 1], :]
    vert_diff_0 = vert_0[neigh[:, 0], :] - vert_0[neigh[:, 1], :]

    R_t = arap_exact(vert_diff_t, vert_diff_0, neigh, n_vert)

    R_neigh_t = 0.5 * (
        torch.index_select(R_t, 0, neigh[:, 0])
        + torch.index_select(R_t, 0, neigh[:, 1])
    )

    vert_diff_0_rot = torch.bmm(R_neigh_t, vert_diff_0.unsqueeze(2)).squeeze()
    acc_t_neigh = vert_diff_t - vert_diff_0_rot

    E_arap = acc_t_neigh.norm() ** 2 + lambda_reg_len * (vert_t - vert_0).norm() ** 2

    return E_arap

In [144]:
class ArapInterpolationEnergy():
    """The interpolation method based on Sorkine et al., 2007"""

    def __init__(self):
        super().__init__()

    # override
    def forward_single(self, vert_new, vert_ref, shape_i):
        E_arap = arap_energy_exact(vert_new, vert_ref, shape_i.get_neigh())
        return E_arap


In [145]:
class ARAPBase(torch.nn.Module):
    def __init__(self, interp_energy):
        super().__init__()
        self.interp_energy = interp_energy
    
    def _loss_deform(self, query_arr_vertices, query_arr_triangles, canonical_arr):
        E = 0
        
        canonical_arr_i = canonical_arr
        E = E + self._loss_deform_single(query_arr_vertices,query_arr_triangles,canonical_arr_i)
        return E
    
    def _loss_deform_single(self, query_vertices, query_faces, canonical_arr_i):
       
        E_deform = 0.0
        
        for i in range(query_vertices.shape[0]):
            
            query_shape = Shape(query_vertices[i],query_faces[i])
            E_x = self.interp_energy.forward_single(
                query_vertices[i],canonical_arr_i , query_shape
            )
            # E_y = self.interp_energy.forward_single(
            #     canonical_arr_i, query_vertices[i], query_shape
            # )

            # E_deform = E_deform + E_x + E_y

        return E_deform

In [146]:
seq_pc = torch.from_numpy(np.load("/usr/stud/srinivaa/code/new_CaDeX/CaDeX/vertices.npz")['points'][0])
seq_triv = torch.from_numpy(np.load("/usr/stud/srinivaa/code/new_CaDeX/CaDeX/faces.npz")['faces'][0])
inputs_cdc = torch.from_numpy(np.vstack(np.load("/usr/stud/srinivaa/code/new_CaDeX/CaDeX/cdc_0.npz")['points']))


In [147]:
inputs_cdc = np.load("/usr/stud/srinivaa/code/new_CaDeX/CaDeX/cdc_0.npz")['points']
inputs_cdc.reshape(-1,3).shape

(117130, 3)

In [148]:
interp_energy = ArapInterpolationEnergy()
arap_base = ARAPBase(interp_energy)
arap_loss = arap_base._loss_deform(seq_pc,seq_triv,inputs_cdc)


Neighbours: tensor([   1,    0,    2,  ..., 3511, 1330, 1330])


IndexError: index 17 is out of bounds for axis 0 with size 17

In [26]:
inputs_cdc = np.load("/usr/stud/srinivaa/code/new_CaDeX/CaDeX/cdc_0.npz")

In [30]:
inputs_cdc.shape



(117130, 3)